In [435]:
import pandas as pd
import numpy as np
import os
import xgboost as xgb
import sklearn.model_selection
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

### Import feature tables

In [17]:
path = os.getcwd() + '/features/'

def fileList():
    matches = []
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            if filename.endswith(('.csv')):
                matches.append(os.path.join(root, filename))
    return matches

list_of_files = fileList()
list_of_files

['/media/anton/DATA/Study/Southampton University/Data Mining/Group Project/features/1000BoWords.csv',
 '/media/anton/DATA/Study/Southampton University/Data Mining/Group Project/features/200BoWords.csv',
 '/media/anton/DATA/Study/Southampton University/Data Mining/Group Project/features/500BoWords.csv',
 '/media/anton/DATA/Study/Southampton University/Data Mining/Group Project/features/50BoWords.csv',
 '/media/anton/DATA/Study/Southampton University/Data Mining/Group Project/features/coefs.csv',
 '/media/anton/DATA/Study/Southampton University/Data Mining/Group Project/features/coefs_May_apr.csv',
 '/media/anton/DATA/Study/Southampton University/Data Mining/Group Project/features/DynamicepertimeFeature.csv',
 '/media/anton/DATA/Study/Southampton University/Data Mining/Group Project/features/just_ass_rules.csv',
 '/media/anton/DATA/Study/Southampton University/Data Mining/Group Project/features/just_top_10.csv',
 '/media/anton/DATA/Study/Southampton University/Data Mining/Group Project/f

### Anton - Lakshmi features

In [410]:
n = [4, 5, 6]
l = []
for idx, filename in enumerate(list_of_files):
    if idx in n:
        data = pd.read_csv(filename)
        l.append(data)
        
for idx, table in enumerate(l):
    if idx == 0:
        ds = table.copy()
    else:
        ds = ds.merge(table, on="itest_id", how="inner")

In [411]:
ds = ds.drop("Unnamed: 0", axis=1)
# ds[pd.isnull(ds).any(axis=1)]
# ds["Difference"] = ds["Difference"].fillna(0)
ds.isnull().sum()

itest_id                0
num_of_q_per_session    0
n_sessions              0
n_questions             0
accuracy                0
Difference              5
isstem                  0
dtype: int64

In [165]:
ds.head()

,itest_id,num_of_q_per_session,n_sessions,n_questions,accuracy,Difference,isstem
0,9,39.142857,7,51.0,0.823529,0.045752,1.0
1,27,8.571429,7,25.0,0.760000,0.303571,0.0
2,33,21.833333,6,3.0,0.666667,0.148148,0.0
3,35,33.071429,14,45.0,0.755556,-0.166667,0.0
4,37,17.000000,3,13.0,0.692308,-0.057692,0.0


### Train model

In [87]:
# xgBoost
# parameters http://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
clf_xgb = xgb.XGBClassifier(max_depth=4, n_estimators=200)
clf_forest = RandomForestClassifier(max_features=3, n_estimators=1000)

# choose classifier
clf = clf_forest

# define data
x = pd.DataFrame(ds.drop(["isstem", "itest_id"], axis=1))
y = ds["isstem"]

# normalize data
scaler = StandardScaler()
x = scaler.fit_transform(x)

# split data
xx_tr, xx_te, yy_tr, yy_te = sklearn.model_selection.train_test_split(x, y, train_size=0.8)
clf.fit(xx_tr, yy_tr)
yy_predicted = clf.predict_proba(xx_te)

t = pd.DataFrame(yy_predicted)
metrics.roc_auc_score(yy_te, t[1])

0.62153759185980773

### Qi - Fay - Joy features

In [412]:
n = [9]
l = []
for idx, filename in enumerate(list_of_files):
    if idx in n:
        data = pd.read_csv(filename)
        l.append(data)
        
for idx, table in enumerate(l):
    if idx == 0:
        df = table.copy()
    else:
        df = df.merge(table, on="itest_id", how="inner")
        
pd.set_option('max_columns', 115)
pd.set_option('max_rows', 115)
df = df[df["itest_id"].isin(ds["itest_id"])]
df.shape

(449, 115)

In [413]:
# df.shape
# len(df['itest_id'].unique())

# k = 0
# for i in df.groupby("itest_id"):
#     k += 1
#     print(i[0])
# k

# if 'isstem' in df.columns:
#     print('TRUE')

# pd.DataFrame([df["itest_id"], ds["itest_id"]])

In [414]:
tmp = ds[["isstem"]]
df = df.reset_index(drop=True)
df = pd.concat([df, tmp], axis=1)
# df.corr()

# HOW TO REPLACE NAN??? <-- need to solve it
df = df.fillna(0)
df.isnull().sum()
# df["isstem"]
df.head()

,itest_id,aveknow,avecarelessness,avecorrect,numactions,averesbored,averesengcon,averesconf,averesfrust,averesofftask,averesgaming,actioncount,bottomhint,correct,correctrate,dayfinishquestion,filedin,frhrate,frscaffoldh,hint,hintrate,monthfinishquestion,scaffold,timeover80,timeover80rate,timetaken,timetohint,workinschoolrate,wrong,wrongrate,actioncount_c1,bottomhint_c1,correct_c1,correctrate_c1,dayfinishquestion_c1,filedin_c1,frhrate_c1,frscaffoldh_c1,hint_c1,hintrate_c1,monthfinishquestion_c1,scaffold_c1,timeover80_c1,timeover80rate_c1,timetaken_c1,timetohint_c1,workinschoolrate_c1,wrong_c1,wrongrate_c1,actioncount_c2,bottomhint_c2,correct_c2,correctrate_c2,dayfinishquestion_c2,filedin_c2,frhrate_c2,frscaffoldh_c2,...,monthfinishquestion_c2,scaffold_c2,timeover80_c2,timeover80rate_c2,timetaken_c2,timetohint_c2,workinschoolrate_c2,wrong_c2,wrongrate_c2,actioncount_c3,bottomhint_c3,correct_c3,correctrate_c3,dayfinishquestion_c3,filedin_c3,frhrate_c3,frscaffoldh_c3,hint_c3,hintrate_c3,monthfinishquestion_c3,scaffold_c3,timeover80_c3,timeover80rate_c3,timetaken_c3,timetohint_c3,workinschoolrate_c3,wrong_c3,wrongrate_c3,average_questions_by_day,average_questions_by_mon,counts_day_answer,counts_mon_answer,std_questions_by_day,std_questions_by_mon,average_questions_by_day_c1,average_questions_by_mon_c1,counts_day_answer_c1,counts_mon_answer_c1,std_questions_by_day_c1,std_questions_by_mon_c1,average_questions_by_day_c2,average_questions_by_mon_c2,counts_day_answer_c2,counts_mon_answer_c2,std_questions_by_day_c2,std_questions_by_mon_c2,average_questions_by_day_c3,average_questions_by_mon_c3,counts_day_answer_c3,counts_mon_answer_c3,std_questions_by_day_c3,std_questions_by_mon_c3,wrong_and_over80,wrong_and_over80_c1,wrong_and_over80_c2,wrong_and_over80_c3,isstem
0,9,0.185138,0.099734,0.438492,504,0.277149,0.644744,0.098078,0.162771,0.213378,0.005554,1.976471,0.054902,0.866667,0.598004,57384.031373,0.000000,0.460008,0.609085,0.419608,0.113438,573.713725,1.078431,0.274510,0.156434,83.988235,16.705882,1.0,0.690196,0.288558,2.054545,0.036364,0.909091,0.602042,57464.981818,0.000000,0.387042,0.580606,0.400000,0.077590,574.545455,1.072727,0.236364,0.133838,84.090909,10.600000,1.0,0.745455,0.320368,2.206897,0.068966,0.931034,0.629930,58709.241379,0.000000,0.472792,0.637931,...,586.982759,1.310345,0.241379,0.084377,86.672414,19.965517,1.0,0.879310,0.301603,1.817308,0.038462,0.817308,0.587592,56764.471154,0.000000,0.474611,0.623397,0.394231,0.131616,567.500000,1.009615,0.259615,0.179670,76.567308,16.365385,1.0,0.605769,0.280792,36.428571,51.000000,7,5,13.624208,14.212670,7.857143,11.000000,7.0,5.0,4.561746,4.743416,8.285714,11.600000,7.0,5.0,6.945365,6.655825,14.857143,20.800000,7.0,5.0,7.515064,4.816638,0.070588,0.018182,0.034483,0.105769,1.0
1,27,0.142031,0.069297,0.348837,129,0.330226,0.551367,0.122658,0.095420,0.348090,0.036900,2.632653,0.204082,0.918367,0.573893,56319.795918,0.000000,0.412679,0.558875,0.775510,0.108286,563.081633,0.979592,0.714286,0.360158,243.408163,51.897959,1.0,0.938776,0.317821,1.200000,0.000000,1.000000,0.900000,52993.200000,0.000000,0.800000,0.800000,0.000000,0.000000,529.800000,0.800000,0.400000,0.300000,63.600000,-1.000000,1.0,0.200000,0.100000,2.727273,0.272727,0.909091,0.592532,54594.818182,0.000000,0.183442,0.327922,...,545.818182,1.000000,1.000000,0.501623,318.090909,110.909091,1.0,1.090909,0.271645,3.240000,0.240000,0.960000,0.476116,57890.600000,0.000000,0.400136,0.591109,1.120000,0.136478,578.800000,1.040000,0.720000,0.297195,279.680000,52.640000,1.0,1.160000,0.387406,8.166667,12.250000,6,4,6.112828,11.786291,2.500000,2.500000,2.0,2.0,2.121320,2.121320,3.666667,5.500000,3.0,2.0,3.055050,2.121320,5.000000,8.333333,5.0,3.0,2.345208,8.386497,0.163265,0.000000,0.181818,0.200000,0.0
2,33,0.459813,0.202787,0.686391,169,0.260426,0.650769,0.059971,0.061834,0.296286,0.010954,1.233577,0.007299,0.846715,0.783212,60214.941606,0.000000,0.431022,0.489051,0.051095,0.012530,602.036496,0.649635,0.328467,0.285158,88.233577,2.9

### Dimensionality reduction

In [159]:
# define data
x = pd.DataFrame(df.drop(["isstem", "itest_id"], axis=1))
y = df["isstem"]

# normalize data
scaler = StandardScaler()
x = scaler.fit_transform(x)

# dimensionality reduction
pca = PCA(n_components=20)
x = pca.fit_transform(x)

# split data
xx_tr, xx_te, yy_tr, yy_te = sklearn.model_selection.train_test_split(x, y, train_size=0.8)

In [164]:
# xgBoost
# parameters http://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
clf_xgb = xgb.XGBClassifier(max_depth=15, n_estimators=1000)
clf_forest = RandomForestClassifier(max_features=5, n_estimators=1000)

# choose classifier
clf = clf_xgb

# fit model
clf.fit(xx_tr, yy_tr)
yy_predicted = clf.predict_proba(xx_te)

t = pd.DataFrame(yy_predicted)
metrics.roc_auc_score(yy_te, t[1])

0.62362102530824137

### Yeldos - Richa BoW features

In [415]:
# n = [0, 1, 2, 3]
n = [0]
l = []
for idx, filename in enumerate(list_of_files):
    if idx in n:
        data = pd.read_csv(filename)
        l.append(data)
        
for idx, table in enumerate(l):
    if idx == 0:
        dp = table.copy()
    else:
        dp = dp.merge(table, on="itest_id", how="inner")
    
dp = dp.drop("Unnamed: 0", axis=1)
dp.shape

(326, 1002)

In [351]:
pd.DataFrame([ds["itest_id"], dp["itest_id"]])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,...,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448
itest_id,9.0,27.0,33.0,35.0,37.0,41.0,77.0,87.0,114.0,126.0,129.0,176.0,190.0,205.0,224.0,237.0,240.0,263.0,269.0,282.0,283.0,303.0,321.0,334.0,360.0,363.0,376.0,387.0,390.0,391.0,403.0,413.0,462.0,473.0,476.0,493.0,511.0,522.0,621.0,636.0,638.0,640.0,661.0,676.0,684.0,688.0,738.0,739.0,747.0,756.0,799.0,808.0,826.0,828.0,832.0,848.0,869.0,...,6615.0,6621.0,6626.0,6627.0,6646.0,6694.0,6695.0,6697.0,6741.0,6751.0,6756.0,6764.0,6782.0,6792.0,6849.0,6862.0,6876.0,6879.0,6911.0,6935.0,6938.0,6939.0,6948.0,6950.0,6952.0,6964.0,6974.0,6984.0,7023.0,7028.0,7057.0,7060.0,7062.0,7074.0,7079.0,7083.0,7093.0,7110.0,7174.0,7179.0,7181.0,7225.0,7233.0,7267.0,7288.0,7289.0,7294.0,7308.0,7355.0,7369.0,7457.0,7511.0,7629.0,7676.0,7680.0,7717.0,7782.0
itest_id,35.0,77.0,126.0,205.0,283.0,360.0,391.0,473.0,476.0,621.0,636.0,661.0,676.0,684.0,688.0,738.0,739.0,747.0,756.0,799.0,808.0,826.0,828.0,832.0,848.0,869.0,887.0,892.0,894.0,898.0,922.0,943.0,964.0,998.0,1046.0,1047.0,1099.0,1107.0,1109.0,1135.0,1177.0,1203.0,1234.0,1238.0,1277.0,1282.0,1301.0,1305.0,1309.0,1348.0,1350.0,1371.0,1375.0,1417.0,1456.0,1460.0,1469.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [352]:
# two[two["itest_id"].isin(dp["itest_id"])]

In [353]:
dp[dp["isstem"] == 0].shape

(253, 1002)

### Dimensionality reduction

In [354]:
# define data
x = pd.DataFrame(dp.drop(["isstem", "itest_id"], axis=1))
# x = pd.DataFrame(dp.drop(["itest_id"], axis=1))
y = dp["isstem"]
# y = temp

# normalize data
scaler = StandardScaler()
x = scaler.fit_transform(x)

# dimensionality reduction
# pca = PCA(n_components=100)
# x = pca.fit_transform(x)

# split data
xx_tr, xx_te, yy_tr, yy_te = sklearn.model_selection.train_test_split(x, y, train_size=0.8)

In [355]:
# xgBoost
# parameters http://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
clf_xgb = xgb.XGBClassifier(max_depth=3, n_estimators=1000)
clf_forest = RandomForestClassifier(max_features=64, n_estimators=500)

# choose classifier
clf = clf_xgb

# fit model
clf.fit(xx_tr, yy_tr)
yy_predicted = clf.predict_proba(xx_te)

t = pd.DataFrame(yy_predicted)
metrics.roc_auc_score(yy_te, t[1])

0.67973856209150318

### Yeldos - Richa rest features

In [416]:
# n = [7, 8, 10]
n = [10]
l = []
for idx, filename in enumerate(list_of_files):
    if idx in n:
        data = pd.read_csv(filename)
        l.append(data)
        
for idx, table in enumerate(l):
    if idx == 0:
        dn = table.copy()
    else:
        dn = dn.merge(table, on="itest_id", how="inner")
    
dn = dn.drop("Unnamed: 0", axis=1)
dn.shape

(326, 100)

In [417]:
# dp["isstem"]
# two[two["itest_id"].isin(dp["itest_id"])]
temp = ds[ds["itest_id"].isin(dn["itest_id"])]
temp = temp.reset_index(drop=True)
dn["isstem"] = temp["isstem"]
dn.head()

,itest_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,ass_71.0,ass_26.0,ass_99.0,ass_40.0,ass_18.0,ass_35.0,isstem
0,35.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0.0
1,77.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0.0
2,126.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0,1,0,0,-1,0,0.0
3,205.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-1,0,0.0
4,283.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,-1,0,0.0
5,360.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0
6,391.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0.0
7,473.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,1,0,-1,-1,0,0.0
8,476.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,-1,0,0,0.0
9,621.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0

### Merge all the features

In [418]:
# ds, df, dp, dn

# if dp and dn are smaller than ds and df
if dp.shape[0] < ds.shape[0]:
    
    # downsample ds and df
    ds = ds[ds["itest_id"].isin(dn["itest_id"])]
    df = df[df["itest_id"].isin(dn["itest_id"])]
    
    # reset index
    ds = ds.reset_index(drop=True)
    df = df.reset_index(drop=True)


ds = ds.drop(["itest_id", "isstem"], axis=1)
df = df.drop(["itest_id", "isstem"], axis=1)
dp = dp.drop(["itest_id", "isstem"], axis=1)
dn = dn.drop(["itest_id"], axis=1)

print(ds.shape)
print(df.shape)
print(dn.shape)
print(dp.shape)




(326, 5)
(326, 114)
(326, 100)
(326, 1000)


In [422]:
X = pd.concat([ds, df, dp, dn.drop("isstem", axis=1)], axis=1, ignore_index=True)
y = dn["isstem"]

In [425]:
# this can be wrong
X = X.fillna(0)

### Dimensionality reduction and scaling

In [433]:
# normalize data
scaler = StandardScaler()
x = scaler.fit_transform(X)

# dimensionality reduction
# pca = PCA(n_components=100)
# x = pca.fit_transform(x)

# split data
xx_tr, xx_te, yy_tr, yy_te = sklearn.model_selection.train_test_split(x, y, train_size=0.8)

### Train model

In [438]:
clf_xgb = xgb.XGBClassifier(max_depth=3, n_estimators=1000)
clf_forest = RandomForestClassifier(max_features=64, n_estimators=500)

# choose classifier
clf = clf_xgb

# fit model
clf.fit(xx_tr, yy_tr)
yy_predicted = clf.predict_proba(xx_te)

t = pd.DataFrame(yy_predicted)

AUC = metrics.roc_auc_score(yy_te, t[1])
print("AUC: " + str(AUC))

accuracy = cross_val_score(clf, x, y)
accuracy = np.mean(accuracy)
print("Accuracy: " + str(accuracy))

AUC: 0.547712418301
Accuracy: 0.748653198653


In [441]:
clf_SVM = SVC()

accuracy_SVM = cross_val_score(clf_SVM, x, y)
accuracy_SVM = np.mean(accuracy_SVM)
print("Accuracy of SVM: " + str(accuracy_SVM))

Accuracy of SVM: 0.776094276094
